In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
import time
import re
from lxml import html
from selenium.webdriver.common.action_chains import ActionChains        # 导入鼠标事件库

Project步骤：
1. 根据给出的用户名、密码、浏览器，登陆排排网
2. 逐个搜索出相关基金的信息，进入正确的url
3. 搜集整理相关信息，并生成Excel表格

*放弃搜集净值数据：加密方式很复杂——每一次刷新都会变化&加密手段和之前不同*

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)         # 启动驱动器

#options = webdriver.EdgeOptions()
#driver = webdriver.Edge(options=options)

# 谷歌浏览器 79和79版本后防止被检测
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})

# 私募排排网：
#url = 'https://www.simuwang.com/user/option'
# 黑翼：
heiyi_url = 'https://dc.simuwang.com/product/HF00003YPP.html'
tldker_url = 'https://dc.simuwang.com/product/HF00009YXJ.html'
driver.get(heiyi_url)       # 加载网站

driver.find_element(By.XPATH,'//button[@class="comp-login-method comp-login-b2"]').click()      #点击账号密码登录
driver.find_element(By.XPATH,'//input[@name="username"]').send_keys('13834596916')              # 输入账号
driver.find_element(By.XPATH,'//input[@type="password"]').send_keys('zxcvbnm1')                 # 输入密码
driver.find_element(By.XPATH,'//span[@class="checkbox"]').click()                               # 勾选同意协议
time.sleep(30) # solved this bug by adding sleep time to make sure the page is loaded
driver.find_element(By.XPATH,'//button[@class="comp-login-btn"]').click()                       # 点击登录

#time.sleep(15)

Obstacles: 
1. We don't know what is the correct way to find the login button, may be by input id, or something else. 
2. What are the possible ways to locate?

How did I solve them?
1. Problems are summarised clearly above.
2. Learn how to locate: compare with given code -> find out how they are doing this
3. Find location: google how to find the html code for the window -> locate what needed by searching (like commmand + F) -> insert into the code in the right way

Mistake：没用遵循源代码的全部指示，没有加 time.sleep(15) ，导致报错！（自以为是）

In [ ]:
time.sleep(30)
page = driver.page_source
soup = BeautifulSoup(page,'html.parser')

In [ ]:
def strat(soup):
    tags = [element.text for element in soup.select("div.line2-tag")]
    tags.remove('私募 ')
    tags.remove('正在运作 ')

    return '、'.join(tags)

In [ ]:
import re

output = soup.select("div.line3-common.flex-h-center")
next_year = output[2].text

def extract_percentage(string):
    # The regular expression pattern for a percentage is a number followed by a '%'
    pattern = r'\d+\.?\d*%'

    # Find all matches of the pattern in the string
    matches = re.findall(pattern, string)

    # Return the matches
    return matches

print(type(extract_percentage(next_year)[0]))

In [ ]:
print('a'+'b')

In [ ]:
prod_info = {}

# 获取基金名称
prod_info['基金名称'] = [soup.select("h1.line1-title-name.ellipsis")[0].text] 

prod_info['管理人'] = soup.select("a.line4-baseinfo-value.blue")[0].text

prod_info['策略'] = strat(soup)

output = soup.select("div.line3-common.flex-h-center")
this_year = output[1].text
all_time = output[2].text

if '--' in this_year:
    prod_info['今年来收益'] = '--'
else:
    prod_info['今年来收益'] = extract_percentage(this_year)   # further adjustment is needed

prod_info['成立来收益'] = extract_percentage(all_time)  # further adjustment is needed

# 其他数据
data = list(soup.select("span.line4-baseinfo-value"))[1:]

names = ['成立以来年化', '成立来回撤', '成立以来夏普', '基金状态', '成立日期', '基金经理', 
         '备案编号', '公司管理规模', '今年以来排名']

for i in range(len(data)):
    prod_info[names[i]] = [data[i].text]


In [ ]:
prod_info = pd.DataFrame(prod_info)

with pd.ExcelWriter('prod_info.xlsx') as excel_writer:
    prod_info.to_excel(excel_writer, sheet_name='Sheet1', index=False)

In [ ]:
print(prod_info)